TypedDict
- dict와 TypedDict의 차이점과 TypedDict가 왜 dict 대신 사용되는지 설명해드리겠습니다.

dict와 TypedDict의 주요 차이점
1. 타입 검사
- dict: 런타임에 타입 검사를 하지 않습니다.
- TypedDict: 정적 타입 검사를 제공합니다. 즉, 코드 작성 시 IDE나 타입 체커가 오류를 미리 잡아낼 수 있습니다.

2. 키와 값의 타입
- dict: 키와 값의 타입을 일반적으로 지정합니다 (예: Dict[str, str]).
- TypedDict: 각 키에 대해 구체적인 타입을 지정할 수 있습니다.

3. 유연성
- dict: 런타임에 키를 추가하거나 제거할 수 있습니다.
- TypedDict: 정의된 구조를 따라야 합니다. 추가적인 키는 타입 오류를 발생시킵니다.

4. TypedDict가 dict 대신 사용되는 이유
- 타입 안정성
  - TypedDict는 더 엄격한 타입 검사를 제공하여 잠재적인 버그를 미리 방지할 수 있습니다.
- 코드 가독성
  - TypedDict를 사용하면 딕셔너리의 구조를 명확하게 정의할 수 있어 코드의 가독성이 향상됩니다.
- IDE 지원
  - TypedDict를 사용하면 IDE에서 자동 완성 및 타입 힌트를 더 정확하게 제공받을 수 있습니다.
- 문서화
  - TypedDict는 코드 자체가 문서의 역할을 하여 딕셔너리의 구조를 명확히 보여줍니다.

In [1]:
# TypedDict와 Dict의 차이점 예시
from typing import Dict, TypedDict

# 일반적인 파이썬 딕셔너리(dict) 사용
sample_dict: Dict[str, str] = {
    "name": "테디",
    "age": "30",  # 문자열로 저장 (dict 에서는 가능)
    "job": "개발자",
}


# TypedDict 사용
class Person(TypedDict):
    name: str
    age: int  # 정수형으로 명시
    job: str


typed_dict: Person = {"name": "셜리", "age": 25, "job": "디자이너"}

In [3]:
# dict의 경우
sample_dict["age"] = 35  # 문자열에서 정수로 변경되어도 오류 없음
sample_dict["new_field"] = "추가 정보"  # 새로운 필드 추가 가능

# TypedDict의 경우
typed_dict["age"] = 35  # 정수형으로 올바르게 사용
typed_dict["age"] = "35"  # 타입 체커가 오류를 감지함
typed_dict["new_field"] = (
    "추가 정보"  # 타입 체커가 정의되지 않은 키라고 오류를 발생시킴
)
typed_dict


{'name': '셜리', 'age': '35', 'job': '디자이너', 'new_field': '추가 정보'}

Annotated
- 이 문법은 타입 힌트에 메타데이터를 추가할 수 있게 해줍니다. 

Annotated를 사용하는 주요 이유
- 추가 정보 제공(타입 힌트) / 문서화
- 타입 힌트에 추가적인 정보를 포함시킬 수 있습니다. 이는 코드를 읽는 사람이나 도구에 더 많은 컨텍스트를 제공합니다.
- 코드에 대한 추가 설명을 타입 힌트에 직접 포함시킬 수 있습니다.

- name: Annotated[str, "이름"]
- age: Annotated[int, "나이"]



In [5]:
from typing import Annotated

name: Annotated[str, "사용자 이름"]
age: Annotated[int, "사용자 나이 (0-150)"]

In [6]:
# Pydantic과 함께 사용
from typing import Annotated, List
from pydantic import Field, BaseModel, ValidationError


class Employee(BaseModel):
    id: Annotated[int, Field(..., description="직원 ID")]
    name: Annotated[str, Field(..., min_length=3, max_length=50, description="이름")]
    age: Annotated[int, Field(gt=18, lt=65, description="나이 (19-64세)")]
    salary: Annotated[
        float, Field(gt=0, lt=10000, description="연봉 (단위: 만원, 최대 10억)")
    ]
    skills: Annotated[
        List[str], Field(min_items=1, max_items=10, description="보유 기술 (1-10개)")
    ]


# 유효한 데이터로 인스턴스 생성
try:
    valid_employee = Employee(
        id=1, name="코봉이", age=30, salary=1000, skills=["Python", "LangChain"]
    )
    print("유효한 직원 데이터:", valid_employee)
except ValidationError as e:
    print("유효성 검사 오류:", e)

# 유효하지 않은 데이터로 인스턴스 생성 시도
try:
    invalid_employee = Employee(
        name="코봉",  # 이름이 너무 짧음
        age=17,  # 나이가 범위를 벗어남
        salary=20000,  # 급여가 범위를 벗어남
        skills="Python",  # 리스트가 아님
    )
except ValidationError as e:
    print("유효성 검사 오류:")
    for error in e.errors():
        print(f"- {error['loc'][0]}: {error['msg']}")


유효한 직원 데이터: id=1 name='코봉이' age=30 salary=1000.0 skills=['Python', 'LangChain']
유효성 검사 오류:
- id: Field required
- name: String should have at least 3 characters
- age: Input should be greater than 18
- salary: Input should be less than 10000
- skills: Input should be a valid list


LangGraph에서의 사용(add_messages)
- add_messages는 LangGraph에서 메시지를 리스트에 추가하는 함수입니다.

In [7]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages


class MyData(TypedDict):
    messages: Annotated[list, add_messages]

In [8]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~", id="2")]

result1 = add_messages(msgs1, msgs2)
print(result1)

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'), AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]


In [9]:
# 동일한 ID를 가진 메시지 대체 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [HumanMessage(content="반갑습니다~", id="1")]

result2 = add_messages(msgs1, msgs2)
print(result2)


[HumanMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='1')]
